# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain cohere googlesearch-python tiktoken pypdf qdrant-client openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 M

In [ ]:
! wget https://www.asianpaints.com/content/dam/annual-report-2023/pdf/Asian%20Paints%20IR22-23.pdf

--2024-03-12 10:02:24--  https://www.asianpaints.com/content/dam/annual-report-2023/pdf/Asian%20Paints%20IR22-23.pdf
Resolving www.asianpaints.com (www.asianpaints.com)... 13.107.246.40, 13.107.213.40, 2620:1ec:bdf::40, ...
Connecting to www.asianpaints.com (www.asianpaints.com)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35547144 (34M) [application/pdf]
Saving to: ‘Asian Paints IR22-23.pdf’

Asian Paints IR22-2 100%[===================>]  33.90M   834KB/s    in 42s     

2024-03-12 10:03:09 (826 KB/s) - ‘Asian Paints IR22-23.pdf’ saved [35547144/35547144]



# Automation

In [ ]:
from googlesearch import search
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import cohere
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.models import Distance, VectorParams
from collections import Counter
import numpy as np
import json
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
tokenizer= tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
  tokens = tokenizer.encode(text,disallowed_special=())
  return len(tokens)

In [ ]:
cohere_client = cohere.Client("TBVEwMdI2gEhGDrSClPYR5P8MZLEaBNJl34EmiRW")

qdrant_client = QdrantClient(
    url="https://68c3bd7c-85e8-4ddd-8da6-6d889578e82b.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="G3knmZ-UwEnAyi27lvmVruwAos2I86KfnLIhu7qdLy0p8iqdGMvg-g",
)

In [ ]:
qdrant_client.create_collection(
    collection_name="VectorEmbeddings",
    vectors_config={
        "text-dense": VectorParams(
            size=1024,
            distance=Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [ ]:
stop_words = set(stopwords.words('english'))
def update_vocab(chunk, vocab):
    clean_chunk = re.sub(r'[^a-zA-Z\s]', ' ', chunk)
    tokens = word_tokenize(clean_chunk)
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    for token in tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [ ]:
def capitalise_split(text):
    pattern = r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])'
    parts = re.split(pattern, text)
    return ' '.join(parts)

In [ ]:
def create_sparse_representation(chunk, vocabulary):
    clean_chunk = re.sub(r'[^a-zA-Z\s]', ' ', chunk)
    tokens = word_tokenize(clean_chunk)
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    token_counts = Counter(tokens)
    indices = []
    values = []
    for token, count in token_counts.items():
        if token in vocabulary:
            indices.append(vocabulary[token])
            values.append(count)
    return indices, values

In [ ]:
companies = [("", "")]

def update_vectordb(companies, intype, year):
  for company, ticker in companies:
    with open('/content/drive/MyDrive/vocabulary/vocab.json', 'r') as f:
      vocab = json.load(f)
    query = f"{company} + ' Annual Report 2023', filetype:pdf"
    results = list(search(query, num_results=3))
    for r in results:
      print(r)

    check = int(input("Which pdf do you want to use (0,1,2): "))

    if check in [0,1,2,3,4,5]:
      loader = PyPDFLoader(results[check])
      pages = loader.load_and_split()
      #pages = [doc for doc in pages if doc.metadata['page'] != 206]
      pages = pages[:-3]

      text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 465,
        chunk_overlap=30,
        length_function=tiktoken_len,
        separators=['\n\n','\n',"(?<=\. )",' ']
      )

      data = text_splitter.split_documents(pages)

      datalist = []
      data_metadata = []
      for i in data:
        datalist.append(f"{company}: {capitalise_split(i.page_content)}")
        data_metadata.append(i.metadata)

      for i, metadata in enumerate(data_metadata):
        metadata['uname'] = ticker
        metadata['uid'] = f'{ticker}-{intype}-{year}-{str(i)}'
        metadata['intype'] = intype
        metadata['year'] = year
        metadata['chunk'] = datalist[i]

      print(data_metadata[i])

      embeddings = cohere_client.embed(
      model="embed-english-v3.0",
      input_type="search_document",
      texts=datalist,
      truncate="None"
      ).embeddings

      for chunk in datalist:
        update_vocab(chunk, vocab)

      sparse_vectors = []
      for chunk in datalist:
        indices, values = create_sparse_representation(chunk, vocab)
        sparse_vectors.append((indices, values))
      print(sparse_vectors[:5])

      with open('/content/drive/MyDrive/vocabulary/vocab.json', 'w') as f:
        json.dump(vocab, f)

      cnt = qdrant_client.count(collection_name="VectorEmbeddings").count

      points = [
      models.PointStruct(
          id=cnt + i,
          vector={
              "text-dense": embeddings[i],
              "text-sparse": models.SparseVector(
                  indices=sparse_vectors[i][0],
                  values=sparse_vectors[i][1]
              )
          },
          payload=data_metadata[i]
      ) for i in range(len(datalist))
      ]

      qdrant_client.upsert(
      collection_name="VectorEmbeddings",
      wait=True,
      points=points
      )

    else:
      print("Emundhi le Inka")

In [ ]:
update_vectordb(companies, "AR", 2023)

{'source': '/content/Annual-Report-FY-2022-2023.pdf', 'page': 455, 'uname': 'APOLLOHOSP', 'uid': 'APOLLOHOSP-AR-2023-902', 'intype': 'AR', 'year': 2023, 'chunk': 'Apollo Hospitals: TOTAL 100%  65,313 100%  8,443 100%  (133) 100%  8,310'}
[([11422, 2307, 22, 21754, 111, 573, 485, 2172, 331], [1, 1, 1, 1, 1, 1, 1, 1, 1]), ([11422, 2307, 95, 203, 253, 6683, 93, 21755, 2254, 2398, 1066, 163, 20642, 5904, 2607, 243, 1989, 2182, 244, 23, 153, 263, 185, 186, 183, 224, 251, 252, 15903, 99, 285, 7672, 286, 231, 8022, 1421, 5050, 287, 680, 12970, 1586, 798, 22], [1, 1, 1, 3, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 7, 1, 2, 3, 2, 2, 1, 1, 1, 1, 5, 5, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]), ([11422, 2307, 21755, 12465, 5754, 15903, 485, 12640, 399, 2398, 2254, 21756, 868, 144, 1486, 69, 534, 1625, 240, 2916, 4415, 4499, 820, 940, 5177, 77, 5938, 1066, 2192, 13, 1882, 15, 3744, 594, 7582, 701, 554, 2055, 9, 331, 368, 3536, 21757, 249, 21758, 13648, 374, 62, 2277, 1198, 317, 3447, 314, 9952, 896,

# Hybrid Search

In [ ]:
with open('/content/drive/MyDrive/vocabulary/vocab.json', 'r') as f:
      vocab = json.load(f)

In [ ]:
query_text = ["What did Dhirubhai Ambani (Founder and chairman) say about the success of Reliance Industries?"]

query_indices, query_values = create_sparse_representation(query_text[0], vocab)
query_dense_vector = cohere_client.embed(
      model="embed-english-v3.0",
      input_type="search_document",
      texts=query_text,
      truncate="None"
      ).embeddings

In [ ]:
query_dense_vector = [float(value) for value in query_dense_vector[0]]

In [ ]:
search_results = qdrant_client.search_batch(
    collection_name="VectorEmbeddings",
    requests=[
        models.SearchRequest(
            vector=models.NamedVector(
                name="text-dense",
                vector=query_dense_vector,
            ),
            limit=10,
        ),
        models.SearchRequest(
            vector=models.NamedSparseVector(
                name="text-sparse",
                vector=models.SparseVector(
                    indices=query_indices,
                    values=query_values,
                ),
            ),
            limit=10,
        ),
    ],
)

In [ ]:
search_results

[[ScoredPoint(id=22, version=0, score=0.56215566, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=500, version=0, score=0.5347984, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=499, version=0, score=0.52889055, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=124, version=0, score=0.5009061, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=484, version=0, score=0.49681717, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=959, version=0, score=0.49555165, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=35, version=0, score=0.49075925, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=369, version=0, score=0.48918778, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=749, version=0, score=0.48808023, payload=None, vector=None, shard_key=None),
  ScoredPoint(id=557, version=0, score=0.48691255, payload=None, vector=None, shard_key=None)],
 [ScoredPoint(id=856, version=0, score=47.0, payload=

In [ ]:
qdrant_client.retrieve(
    collection_name="VectorEmbeddings",
    ids=[7400],
)

[Record(id=7400, payload={'chunk': 'Apollo Hospitals: Annual Repor t FY 2022-2023\nLeveraging Technology for Personalised Healthcare', 'intype': 'AR', 'page': 0, 'source': '/content/Annual-Report-FY-2022-2023.pdf', 'uid': 'APOLLOHOSP-AR-2023-0', 'uname': 'APOLLOHOSP', 'year': 2023}, vector=None, shard_key=None)]

# Text Generation

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    openai_api_key = '',
    model_name='gpt-3.5-turbo',
    temperature=0
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
memory_key="chat_history",
k=5,
return_messages=True
)

In [ ]:
def sparse_encoder(chunk:str):
    with open('/content/drive/MyDrive/vocabulary/vocab.json', 'r') as f:
      vocab = json.load(f)
    clean_chunk = re.sub(r'[^a-zA-Z\s]', ' ', chunk)
    tokens = word_tokenize(clean_chunk)
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    token_counts = Counter(tokens)
    indices = []
    values = []
    for token, count in token_counts.items():
        if token in vocab:
            indices.append(vocab[token])
            values.append(count)
    return tuple([indices, values])

In [ ]:
from langchain_community.retrievers import (
    QdrantSparseVectorRetriever,
)

sparse_retriever = QdrantSparseVectorRetriever(
    client=qdrant_client,
    collection_name="VectorEmbeddings",
    sparse_vector_name="text-sparse",
    sparse_encoder=sparse_encoder,
    content_payload_key = "chunk",
    metadata_payload_key = "payload"

)

In [ ]:
sparse_retriever.get_relevant_documents("Asian Paints balance sheet")

[Document(page_content='Asian Paints: Notes to the Consolidated Financial Statements (Contd.)Asian Paints Limited\n466\nIntegrated Annual Report 2022-23 (` in Crores) \nName of the Entity Name of Struck off Company Nature of \ntransactions with \nstruck-off Company As at  \n31.03.2023As at  \n31.03.2022\nAsian Paints Limited Fairgrowth Financial Services Limited Unclaimed Dividend  #  # \nAsian Paints Limited Fairgrowth Investments Limited Unclaimed Dividend  #  # \nAsian Paints Limited Fairtrade Securities Limited Unclaimed Dividend  #  # \nAsian Paints Limited Kinnari Investments Private Limited Unclaimed Dividend  0.01  0.01 \nAsian Paints Limited Mulraj Holdings & Finance Private Limited Unclaimed Dividend  #  # \nAsian Paints Limited Optimist Finvest and Trading Private \nLimited Unclaimed Dividend  #  # \nAsian Paints Limited Palkhi Investment and Trading Company Private Limited Unclaimed Dividend  #  # \nAsian Paints Limited Pax Holdings Private Limited Unclaimed Dividend  #  # 

In [ ]:
from langchain_community.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key = 'TBVEwMdI2gEhGDrSClPYR5P8MZLEaBNJl34EmiRW', model="embed-english-v3.0")

In [ ]:
from langchain_community.vectorstores import Qdrant
qdrant = Qdrant(qdrant_client,"VectorEmbeddings", embeddings, vector_name = "text-dense",content_payload_key = "chunk",
    metadata_payload_key = "payload")

In [ ]:
dense_retriever = qdrant.as_retriever(
    search_type = "similarity"
)

In [ ]:
dense_retriever.get_relevant_documents("Asian Paints Balance sheet")

[Document(page_content='Asian Paints: Asian Paints Limited\n382\nIntegrated Annual Report 2022-23Notes to the Consolidated Financial Statements  \nfor the year ended 31st March, 2023\nGROUP’S BACKGROUN D\nThe Consolidated Financial Statements comprise financial \nstatements of Asian Paints Limited (‘the Parent’ or ‘the \nCompany’) and its subsidiaries (collectively, ‘the Group’) and includes share of profit of the Associates for the year \nended 31\nst March, 2023.\nThe Parent is a public limited Company domiciled and \nincorporated in India under the Indian Companies Act, \n1913. The registered office of the Parent is located at 6A, \nShantinagar, Santacruz East, Mumbai, India.\nThe Group is engaged in the business of manufacturing, \nselling and distribution of paints, coatings, products \nrelated to home décor, bath fittings, modular kitchen & accessories and providing related services.\n1. SIGNIFICANT ACCOUNTING POLICIES \nAND KEY ACCOUNTING ESTIMATES AND \nJUDGEMENTS\n Significant

In [ ]:
from langchain.retrievers import MergerRetriever

lotr = MergerRetriever(retrievers=[dense_retriever, sparse_retriever])

In [ ]:
retrieved_documents = lotr.get_relevant_documents("Balance sheet for Reliance Industries")

In [ ]:
retrieved_documents

[Document(page_content='Reliance Industries: BALANCE SHEET  \nAs at 31st March, 2023\n(C in crore)\nNotes As at  \n31st March 2023As at  \n31st March 2022\nAssets\nNon-Current Assets\nProperty, Plant and Equipment 1 2,32,238 2,23,824\nIntangible Assets 1 12,926 15,802\nCapital Work-in-Progress 1 30,958 19,267\nIntangible Assets Under Development 1 17,957 15,395\nFinancial Assets\nInvestments 2 3,03,558 3,30,493\nLoans 3 22,448 41,951\nOther Financial Assets 4 2,215 2,247\nOther Non-Current Assets 5  2,333 7,297\nTotal Non-Current Assets 6,24,633 6,56,276\nCurrent Assets\nInventories 6 48,926 45,923\nFinancial Assets\nInvestments 7 86,074 78,304\nTrade Receivables 8 16,898 14,394\nCash and Cash Equivalents 9  56,811 21,714\nLoans 10 595 161\nOther Financial Assets 11  49,408 54,901\nOther Current Assets 13 7,220 7,001\nTotal Current Assets 2,65,932 2,22,398\nTotal Assets 8,90,565 8,78,674(C in crore)\nNotes As at  \n31st March 2023As at  \n31st March 2022\nEquity and Liabilities\nEquity

In [ ]:
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
llm,
retriever=lotr,
memory=memory
)

In [ ]:
question = "What did Dhirubhai Ambani say about success of Reliance Industries"""
result = qa({"question": question})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
result['answer']

'Dhirubhai Ambani, the Founder Chairman of Reliance Industries, stated, "Between my past, the present and the future, there is one common factor: Relationship and Trust. This is the foundation of our growth." This quote emphasizes the importance of relationships and trust in the success of Reliance Industries.'

In [ ]:
question = "Is he the founder of this company??"
result = qa({"question": question})

In [ ]:
result['answer']

'Yes, Dhirubhai Ambani is indeed the founder of Reliance Industries.'

In [ ]:
question = "Which digital platform did Reliance Industries launch in Mumbai??"
result = qa({"question": question})

# Qdrant

In [ ]:
import cohere

from qdrant_client.http.models import Batch

cohere_client = cohere.Client("xPQlz92wRppOVGZCHNETnpZhtN9o2ZfO2QEVWh22")

from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://68c3bd7c-85e8-4ddd-8da6-6d889578e82b.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="G3knmZ-UwEnAyi27lvmVruwAos2I86KfnLIhu7qdLy0p8iqdGMvg-g",
)

In [ ]:
from qdrant_client.models import Distance, VectorParams
qdrant_client.create_collection(collection_name="VectorEmbeddings", vectors_config=
    VectorParams(
        size=1024,
        distance=Distance.COSINE,
    )
)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'

In [ ]:
from qdrant_client.models import Distance, VectorParams
qdrant_client.recreate_collection(collection_name="TestCollection", vectors_config=
    VectorParams(
        size=1024,
        distance=Distance.COSINE,
    ),
    sparse_vectors_config={
        "text": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np

embeddings = cohere_client.embed(
    model="embed-english-v3.0",  # New Embed v3 model
    input_type="search_document",  # Input type for documents
    texts=datalist,
    truncate='None'
).embeddings

# Prepare your points with IDs, vectors, and metadata as payload
points = points=[
    models.PointStruct(
        id=i,
        vector=embeddings[i],  # Your vector data here
        payload=data_metadata[i]  # Your metadata here
    ) for i in range(len(datalist))
]

# Perform the upsert operation
qdrant_client.upsert(
    collection_name="TestCollection",
    wait=True,  # Wait for the operation to complete
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
pages[-1:]

[Document(page_content='Asian Paints is India’s leading paint and de ´cor company\nwith a group turnover of D 345 billion. Asian Paints along with its \nsubsidiaries operates in 15 countries and has 27 paint manufacturing facilities in the world serving consumers in 60+ countries. Besides Asian Paints, the group operates across the world through its various brands viz. Asian Paints Berger, Apco Coatings, SCIB Paints, Taubmans, Asian Paints Causeway and Kadisco Asian Paints. Asian Paints also oﬀers a wide range of Home Décor products and is an emerging strong player in the Home Décor space in India.Asian Paints Limited\n6A, Shantinagar, Santacruz (East),Mumbai - 400 055, India. CIN: L24220MH1945PLC004598.investor.relations@asianpaints.com\nwww.asianpaints.com', metadata={'source': '/content/Asian Paints IR22-23.pdf', 'page': 244})]

In [ ]:
qdrant_client.search(
    collection_name="TestCollection",
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  # New Embed v3 model
        input_type="search_query",  # Input type for search queries
        texts=["How many Manufacturing facilities does asian paints maintain?"],
    ).embeddings[0],
)

[ScoredPoint(id=1154, version=1, score=0.7302488, payload={'page': 244, 'source': '/content/Asian Paints IR22-23.pdf', 'uid': 'ASIANPAINTS-AR-2022-2023-1154'}, vector=None, shard_key=None),
 ScoredPoint(id=84, version=1, score=0.6782207, payload={'page': 34, 'source': '/content/Asian Paints IR22-23.pdf', 'uid': 'ASIANPAINTS-AR-2022-2023-84'}, vector=None, shard_key=None),
 ScoredPoint(id=201, version=1, score=0.6638369, payload={'page': 67, 'source': '/content/Asian Paints IR22-23.pdf', 'uid': 'ASIANPAINTS-AR-2022-2023-201'}, vector=None, shard_key=None),
 ScoredPoint(id=283, version=1, score=0.65380675, payload={'page': 81, 'source': '/content/Asian Paints IR22-23.pdf', 'uid': 'ASIANPAINTS-AR-2022-2023-283'}, vector=None, shard_key=None),
 ScoredPoint(id=17, version=1, score=0.6529378, payload={'page': 9, 'source': '/content/Asian Paints IR22-23.pdf', 'uid': 'ASIANPAINTS-AR-2022-2023-17'}, vector=None, shard_key=None),
 ScoredPoint(id=11, version=1, score=0.64885545, payload={'page': 

In [ ]:
len(datalist)

610